In [66]:
!pip install pymysql > /dev/null

In [67]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql (3).json


In [68]:
import json
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)
config['database'] = 'mcdb'
import pymysql
conn = pymysql.connect(
    host=config['host'], 
    user=config['user'], 
    password=config['password'], 
    database=config['database'], 
    port=config['port']
)

In [69]:
import random
random.seed(2021)

In [70]:
cur = conn.cursor()
products_table = '''
      create table if not exists products1 (
        pid INT PRIMARY KEY AUTO_INCREMENT, 
        pname varchar(20) not null,
        pprice INT not null,
        pcategory varchar(20) not null,
        pcost INT not null
        ) AUTO_INCREMENT=11;
'''

In [71]:
cur.execute(products_table)

0

In [72]:
# INSERT INTO table_name (column1, column2, column3,...) VALUES (value1, value2, value3,...)


In [73]:
category_names = ['가구','잡화','벽지']
furn_goods = ['대나무 스툴','벽돌 우물','박스침대', '벚나무 가지', '흐르는 국수장치', '버터 교반기', '물병자리 항아리', '순록 일루미네이션', '아이언 쉘프', '솜사탕 노점']
stuff_goods = ['엄마의 티슈 케이스', '유리컵 양초', '캣그라스', '이끼볼', '아로마스틱', '햄스터철장', '안수리움', '빙고머신', '북엔드' '피크닉 가방']
wall_goods = ['핑크 강판 벽', '퍼플 도트 벽지', '장난기 넘치는 벽지', '갈색 다이너 벽', '구름 위 벽', '딸기 초콜릿 벽', '울창한 숲 벽', '장작더미 벽', '컬러풀 타일 벽','슈퍼마리오 벽']

In [74]:
for item in furn_goods:
    price = random.randint(100,999) * 100
    rate = random.randint(85, 98)
    cost = int(round(price * rate / 1000) * 10)
    print(item, price, '가구', cost)

대나무 스툴 95600 가구 87000
벽돌 우물 97800 가구 92910
박스침대 65700 가구 58470
벚나무 가지 35300 가구 33540
흐르는 국수장치 13500 가구 12420
버터 교반기 58500 가구 54990
물병자리 항아리 16500 가구 14850
순록 일루미네이션 37600 가구 33460
아이언 쉘프 58600 가구 50400
솜사탕 노점 26500 가구 23060


In [75]:
sql = "insert into products1(pname,pprice,pcategory,pcost) values(%s,%s,%s,%s);"
for i, item_list in enumerate([furn_goods, stuff_goods, wall_goods]):
    for item in item_list:
        price = random.randint(100,999) * 100
        rate = random.randint(85, 98)
        cost = int(round(price * rate / 1000) * 10)
        cur.execute(sql, (item,price,category_names[i],cost))
conn.commit()

- Sales 테이블

In [86]:
cur = conn.cursor()
sql = """
    CREATE TABLE if NOT EXISTS sales2(
        sid INT PRIMARY KEY AUTO_INCREMENT,
        sdate DATE NOT NULL,
        scompany VARCHAR(20) NOT NULL,
        spid INT NOT NULL,
        sunit INT NOT NULL,
        FOREIGN KEY(spid) REFERENCES products(pid)
    ) AUTO_INCREMENT=101;
"""

In [ ]:
cur.execute(sql)

In [81]:
calendar = {1:31,2:29,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
company_list = ['도도항공','너굴상점','에이블시스터즈','패트릭상점','여욱마켓']
for i in range(10):
    date = f'2020-1-{random.randint(1,31)}'
    company = company_list[random.randint(0,4)]
    pid = random.randint(11,40)
    unit = random.randint(1,20)
    print(date,company,pid,unit)

2020-1-12 에이블시스터즈 12 9
2020-1-3 도도항공 16 19
2020-1-15 너굴상점 29 5
2020-1-21 패트릭상점 23 9
2020-1-14 너굴상점 31 14
2020-1-5 여욱마켓 37 16
2020-1-30 에이블시스터즈 14 12
2020-1-1 패트릭상점 40 18
2020-1-10 에이블시스터즈 31 4
2020-1-3 여욱마켓 13 14


In [ ]:
sql = "insert into sales1 values(default, %s,%s,%s,%s);"
for month, day in calendar.items():
    for i in range(10):
        date = f'2020-{month}-{random.randint(1,day)}'
        company = company_list[random.randint(0,4)]
        pid = random.randint(11,40)
        unit = random.randint(1,20)
        cur.execute(sql, (date, company, pid, unit))
conn.commit()

- join

In [88]:
sql = """
    CREATE VIEW sales_book as
        SELECT sid, sdate, scompany, pid, pname, pcategory, pprice, sunit, pcost,
            pprice*sunit AS revenue, (pprice-pcost)*sunit AS profit
        FROM sales
        JOIN products
        ON sales.spid = products.pid
        ORDER BY sdate;
"""

In [ ]:
cur = conn.cursor()
cur.execute(sql)

- 데이터를 sales_book.csv로 저장

In [ ]:
sql = 'select * from sales_book'
cur.execute(sql)
results = cur.fetchall()

In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df.head()

In [ ]:
del df[3]
del df[0]

In [ ]:
df.columns = ['판매일자','판매처','상품명','카테고리','단가','수량','원가','매출액','이익']
df.head()

In [ ]:
df.to_csv('sales_book.csv', index=False)

In [ ]:
cur.close()
conn.close()